In [1]:
%autoreload 2 
%matplotlib inline

In [2]:
from __future__ import (print_function, unicode_literals, 
                        division, absolute_import)

In [3]:
import h5py
import pandas as pd
import numpy as np 

In [4]:
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

In [5]:
import healpy as hp

In [6]:
import extract_catalog as ext_cat
import get_gal_centroids as getg
import plot_clst_prop as plotClst
from IPython.html.widgets import interact

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [7]:
dataPath = "../../data/"

In [8]:
original_f = h5py.File(dataPath + 
                "Illustris-1_fof_subhalo_myCompleteHaloCatalog_00135.hdf5")

In [49]:
clstNo = 4

In [9]:
df = ext_cat.extract_clst(original_f, clstNo)

In [10]:
pos = [k for k in df.keys() if "SubhaloPos" in k]

# nsides of HEALpix are powers of 2

In [41]:
xi_array, phi_array = getg.angles_given_HEALpix_nsides(2)

# Start projection!

In [42]:
df.shape[0]

7573

In [43]:
cut_kwargs={"DM_cut": 1e3, "star_cut": 1e2}

mask = getg.cut_reliable_galaxies(df, **cut_kwargs)

In [44]:
col = ["SubhaloPos{0}".format(i) for i in range(3)]

data = np.array(df[col][mask])

In [64]:
data_arr, xi_array, phi_array, los_axis = \
    getg.prep_data_with_cuts_and_proj(df, getg.cut_reliable_galaxies, 
                                      cut_kwargs, nside_pow=1, 
                                      verbose=True)

# of subhalos after the cut = 265


In [72]:

map(lambda d, xi, phi: 
    getg.compute_KDE_peak_offsets(d, original_f, clstNo, xi=xi, 
                                  phi=phi, los_axis=los_axis),
    data_arr[:1], xi_array[:1], phi_array[:1])

[[9.9493751986735788,
  0.012427017975398531,
  {'bandwidth_matrix_H': array([[ 26556.13637187,   1862.10888918],
          [  1862.10888918,  24949.06176464]]),
   'estimate': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
   'eval_points': array([[-1615.71111087, -1592.7716575 , -1569.83220413, -1546.89275077,
           -1523.9532974 , -1501.01384403, -1478.07439067, -1455.1349373 ,
           -1432.19548393, -1409.25603057, -1386.3165772 , -1363.37712383,
           -1340.43767047, -1317.4982171 , -1294.55876373, -1271.61931037,
           -1248.679857  , -1225.74040363, -1202.80095027, -1179.8614969 ,
           -1156.92204353, -1133.98259017, -1111.0431368 , -1088.10368343,
           -1065.16423007, -1042.2247767 , -1019.28532333,  -